In [1]:
# !pip install selenium
# !pip install webdriver_manager
from datetime import datetime, timedelta
from random import randrange
from afip import save_data_to_file, read_data_from_file, AFIP
import os

In [8]:
def make_new_data_file():
    if os.path.exists('data.json'):
        print('Delete data.json first!!!') 
        return

    # Initial conditions
    start_date = datetime(2025, 5, 1)  # Start
    total_days = 31  # Number of days in December
    holidays = {1}  # Holidays
    total_amount = randrange(2700, 3000)  # Total amount in thousands
    maximum_day_amount = 172000
    minimum_day_amount = 150000
    data = []  # Initial list
    daily_amounts = []  # List to store amounts for each working day

    # Check when December starts (on Friday in 2023)
    weekday = start_date.weekday()

    # Generate amounts for working days, accounting for holidays
    for day in range(1, total_days + 1):
        current_date = start_date + timedelta(days=day - 1)
        # Check if the day is a working day (Mon-Fri) and not a holiday
        if current_date.weekday() < 5 and current_date.day not in holidays:
            # Generate amounts that are multiples of thousands and do not exceed 39000
            daily_amounts.append(randrange(25, 40) * 1000)

    # Algorithm to distribute amounts to keep the total amount within the specified bounds
    # This may require multiple attempts and adjustments based on the number of working days
    while sum(daily_amounts) != total_amount * 1000:
        for i in range(len(daily_amounts)):
            # Make adjustments considering the maximum amount
            if sum(daily_amounts) < total_amount * 1000 and daily_amounts[i] < maximum_day_amount:
                daily_amounts[i] += 1000
            elif sum(daily_amounts) > total_amount * 1000 and daily_amounts[i] > minimum_day_amount:
                daily_amounts[i] -= 1000

    # Populate the final list with data
    current_day = 0
    for day in range(1, total_days + 1):
        current_date = start_date + timedelta(days=day - 1)
        if current_date.weekday() < 5 and current_date.day not in holidays:
            data.append(dict(date=current_date.strftime('%d/%m/%Y'), price=str(daily_amounts[current_day])))
            current_day += 1

    # Display the list
    summ = 0
    for i, entry  in enumerate(data):
        summ += int(entry['price'])
        print(f'{i}  {entry}')
    print(summ)
    save_data_to_file(data, 'data.json')

make_new_data_file()

0  {'date': '02/05/2025', 'price': '128000'}
1  {'date': '05/05/2025', 'price': '135000'}
2  {'date': '06/05/2025', 'price': '136000'}
3  {'date': '07/05/2025', 'price': '129000'}
4  {'date': '08/05/2025', 'price': '135000'}
5  {'date': '09/05/2025', 'price': '134000'}
6  {'date': '12/05/2025', 'price': '141000'}
7  {'date': '13/05/2025', 'price': '128000'}
8  {'date': '14/05/2025', 'price': '128000'}
9  {'date': '15/05/2025', 'price': '138000'}
10  {'date': '16/05/2025', 'price': '131000'}
11  {'date': '19/05/2025', 'price': '138000'}
12  {'date': '20/05/2025', 'price': '132000'}
13  {'date': '21/05/2025', 'price': '127000'}
14  {'date': '22/05/2025', 'price': '133000'}
15  {'date': '23/05/2025', 'price': '132000'}
16  {'date': '26/05/2025', 'price': '133000'}
17  {'date': '27/05/2025', 'price': '137000'}
18  {'date': '28/05/2025', 'price': '133000'}
19  {'date': '29/05/2025', 'price': '128000'}
20  {'date': '30/05/2025', 'price': '130000'}
2786000


In [2]:
data = read_data_from_file('data.json')
summ = 0
for i, entry  in enumerate(data):
    summ += int(entry['price'])
    print(f'{i}  {entry}')
print(summ)

0  {'date': '02/05/2025', 'price': '128000'}
1  {'date': '05/05/2025', 'price': '135000'}
2  {'date': '06/05/2025', 'price': '136000'}
3  {'date': '07/05/2025', 'price': '129000'}
4  {'date': '08/05/2025', 'price': '135000'}
5  {'date': '09/05/2025', 'price': '134000'}
6  {'date': '12/05/2025', 'price': '141000'}
7  {'date': '13/05/2025', 'price': '128000'}
8  {'date': '14/05/2025', 'price': '128000'}
9  {'date': '15/05/2025', 'price': '138000'}
10  {'date': '16/05/2025', 'price': '131000'}
11  {'date': '19/05/2025', 'price': '138000'}
12  {'date': '20/05/2025', 'price': '132000'}
13  {'date': '21/05/2025', 'price': '127000'}
14  {'date': '22/05/2025', 'price': '133000'}
15  {'date': '23/05/2025', 'price': '132000'}
16  {'date': '26/05/2025', 'price': '133000'}
17  {'date': '27/05/2025', 'price': '137000'}
18  {'date': '28/05/2025', 'price': '133000'}
19  {'date': '29/05/2025', 'price': '128000'}
20  {'date': '30/05/2025', 'price': '130000'}
2786000


In [3]:
summ = 0
data = data[13:18]
for i, entry  in enumerate(data):
    summ += int(entry['price'])
    print(f'{i}  {entry}')
print(summ)

0  {'date': '21/05/2025', 'price': '127000'}
1  {'date': '22/05/2025', 'price': '133000'}
2  {'date': '23/05/2025', 'price': '132000'}
3  {'date': '26/05/2025', 'price': '133000'}
4  {'date': '27/05/2025', 'price': '137000'}
662000


In [4]:
################ DANGER!!!!!!!!!!!!!!!!!!!!!!!
wp = AFIP()
for el in data:
    print(f"making invoice on {el['date']} ({el['price']} pesos)...")
    wp.make_invoice(date=el['date'], price=el['price'])
print('done!')
wp.close()

start login
find element //a[@href="https://auth.afip.gob.ar/contribuyente_/login.xhtml"]
find username field
find element //input[@id="F1:username"]
find element //input[@id="F1:username"]
find element //input[@id="F1:btnSiguiente"]
find password field
find element //input[@id="F1:password"]
find element //input[@id="F1:password"]
find element //input[@id="F1:btnIngresar"]
find element //a[.//div[.//h3[text()="Comprobantes en línea"]]]
find element //input[@value="NAZAROV SERGEI"]
Модальное окно не найдено за 3 секунды.
making invoice on 21/05/2025 (127000 pesos)...
find element //a[@id="btn_gen_cmp"]
Модальное окно не найдено за 3 секунды.
find element //select[@name="puntoDeVenta"]
find element //input[@value="Continuar >"]
find element //input[@id="fc"]
find element //select[@id="idconcepto"]
find element //input[@id="fsd"]
find element //input[@id="fsh"]
find element //input[@id="vencimientopago"]
find element //input[@value="Continuar >"]
find element //select[@id="idivareceptor"